# NoPE - Event-Distributor

The NoPE-Dispatcher uses one `eventDistributor`. The distributor is used to share the events accross to other distributors.

## An Event

A event is not persisent. Once it is fired the `event-listeners` will be informed. There is ***no*** historic access to events. If you subscribed after the Event has been fired, you wont get 

## 

In [1]:
// First lets install nope using npm
const nope = require("../dist-nodejs/index.nodejs")

// Create a communicator:
// We will use the event layer (which just runs internally)
const communicator = nope.getLayer("event");

// Lets create our dispatcher

// 1. Dispatcher simulates our local system
const localDispatcher = nope.dispatcher.getDispatcher({
  communicator,
  id: "local"
}, {
  singleton: false,
  useBaseServices: false
});

> For Jupyter we need an extra async wrapper to wait for initalizing the dispatcher:

see here for the details in Jupyter: https://n-riesco.github.io/ijavascript/doc/async.ipynb.html

In [2]:
$$.async();
// Lets wait for our element to be ready.
localDispatcher.ready.waitFor().then($$.done);

true

Now we want to listen to newly connected dispatchers. For this purpose, we create an observer, which will listen to changes.

In [3]:
localDispatcher.eventDistributor.emit("eventName","eventData");

Additionally we want to show the currently connected dispatchers. In this data the own dispatcher will **allways** be included:

In [4]:
// Show our connected Dispatchers
let connectedDispatchers = localDispatcher.connectivityManager.dispatchers.data.getContent();
let localDispatcherIncluded = connectedDispatchers.includes(localDispatcher.id);

// Now lets log our results.
console.log("connectedDispatchers    =", connectedDispatchers);
console.log("localDispatcherIncluded =", localDispatcherIncluded);

connectedDispatchers    = [ 'local' ]
localDispatcherIncluded = true


Now that we have implemented our listeners and have seen the connected dispatchers (which is only the `"local"`-dispatchre), We will add an additional dispatcher. This should result in calling our `onChange`-listener. Additionally, we wait until our `remoteDispatcher` is initalized

In [5]:
// 2. Dispatcher simulates our remote system
const remoteDispatcher = nope.dispatcher.getDispatcher({
  communicator,
  id: "remote"
}, {
  singleton: false,
  useBaseServices: false
});


Now we want to see, which system is the current master. This should be our `local`.

In [6]:
// We expect to be the master, because the localDispatcher has been created first.
console.log("master =", localDispatcher.connectivityManager.master.id);

master = local


Now we want to use our `eventDistributor`. Therefore we will create different subscriptions:

In [7]:
const observers = []

observers.push(
    localDispatcher.eventDistributor.registerSubscription('#', (data, rest) => {
        console.log("1. '#'          received " + JSON.stringify(data) + " - ", rest)
    })
);

observers.push(
    localDispatcher.eventDistributor.registerSubscription('test/+', (data, rest) => {
        console.log("2. 'test/+'     received " + JSON.stringify(data) + " - ", rest)
    })
);

observers.push(
    localDispatcher.eventDistributor.registerSubscription('+/event/someProp', (data, rest) => {
        console.log("3. '+/event'    received " + JSON.stringify(data) + " - ", rest)
    })
);

observers.push(
    localDispatcher.eventDistributor.registerSubscription('some/event', (data, rest) => {
        // => Should not be called.
        console.log("4. 'some/event' received " + JSON.stringify(data) + " - ", rest)
    })
);


remoteDispatcher.eventDistributor.emit("test/event", {someProp: "Hello Local Dispatcher!"});


1. '#'          received {"someProp":"Hello Local Dispatcher!"} -  {
  timestamp: 1680018177281,
  forced: false,
  args: [],
  sender: '3b895326-7460-4bab-83c6-f676a9e8006a',
  pubSubUpdate: true,
  topicOfChange: 'test/event',
  topicOfContent: 'test/event',
  topicOfSubscription: '#'
}
2. 'test/+'     received {"someProp":"Hello Local Dispatcher!"} -  {
  timestamp: 1680018177281,
  forced: false,
  args: [],
  sender: '3b895326-7460-4bab-83c6-f676a9e8006a',
  pubSubUpdate: true,
  topicOfChange: 'test/event',
  topicOfContent: 'test/event',
  topicOfSubscription: 'test/+'
}


UnhandledPromiseRejection: This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). The promise rejected with the reason "The Path contains wildcards. Please use the method "patternbasedPullData" instead".